# Vector search with LangChain (Azure AI Search)
이 노트북에서는 LangChain을 통해 Azure AI Search의 검색 기능들을 사용해 볼 것이다.

- 키워드 검색
- 벡터 검색
- 하이브리드 검색
- 시맨틱 하이브리드 검색



# 사전 준비
이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:
- Azure AI Search 리소스의 엔드포인트 및 쿼리 API 키
- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service에 배포된 `text-embedding-ada-002` Embeddings 모델. 이 모델의 API 버전은 `2023-05-15`을 사용했다. 배포 이름은 모델 이름과 동일하게 `text-embedding-ada-002`로 명명했다.
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI Embeddings 모델의 배포 이름
  - OpenAI API 버전
- Python (이 예제는 버전 3.10.x로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.

## 패키지 설치

<div class="alert alert-block alert-success">
<b>Note:</b> LangChain의 pull request <a href="https://github.com/langchain-ai/langchain/pull/14789">#14789</a>가 릴리즈에 포함되기 전까지는 azure-search-documents==11.4.0b8를 사용해야 한다.</div>


In [ ]:
!pip uninstall -y azure-search-documents
!pip install azure-search-documents==11.4.0b8
#pip install azure-search-documents==11.4.0

In [ ]:
!pip install openai[datalib]==1.3.9
!pip install langchain==0.0.350

In [ ]:
import azure.search.documents
print("azure.search.documents", azure.search.documents.__version__)
import openai
print("openai", openai.__version__)
import langchain
print("langchain", langchain.__version__)

## 라이브러리 및 환경변수 불러오기

In [ ]:
import os

os.environ["AZURESEARCH_FIELDS_CONTENT"]="content"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"]="embedding"

from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

## 연동 설정

In [ ]:
vector_store_address: str = "<Your search service endpoint>"
vector_store_password: str = "<Your search service query key>"

os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

model: str = "embedding" # 자동 구축시 기본으로 설정
index_name: str = "gptkbindex" # 자동 구축시 기본으로 설정


In [ ]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=model,
    openai_api_version="2024-02-01"
)

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name="default"
)

# 1. キーワード検索
最もシンプルなキーワード検索のクエリーです。`ja.lucene` というスタンダードな日本語アナライザーに搭載されている辞書ベースのトークナイザーによって、これらのトークンに分解されます。このトークンを用いて転置インデックスが構築されます。

次に TF/IDF ベースの [BM25](https://ja.wikipedia.org/wiki/Okapi_BM25) スコアリングアルゴリズムによって、文章中からトークンの一致頻度を見て関連性スコアが決定されます。細かくはトークンのレア度や文章中の密度なども重みづけされます。キーワード検索では、わざとスペルミスしたので、**源実朝**という一つのトークンにならずに、**一文字ずつのトークンになって関係ない人名の部分でヒット**してしまったりしてますね。また、「特徴」というワードをそのまま検索しており、文章中に「和歌の特徴」とあからさまに書いていない限りこのワードは意味を成しません。残念ながらこの検索方法では、「和歌の特徴を知りたい」というユーザーの意図は考慮されません。

In [ ]:
from langchain.retrievers import AzureCognitiveSearchRetriever

query = "源実友のお歌にはどのような特徴があったのでしょうか？"
retriever = AzureCognitiveSearchRetriever(
    service_name="gptkb-xxxxxxxxx", #Your Azure AI Search service name
    index_name=index_name,
    api_key=vector_store_password,
    content_key="content",
    top_k=3,
)

docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)
    

# 2. 벡터 유사도 검색

# 2.1. 간단한 벡터 검색
'최**무**가 간행에 기여한**경전**에는 어떤 특징이 있나요?'처럼 일부러 철자를 틀리게 입력하고 불교 경전을 경전이라고만 작성한 쿼리로 검색을 해보자. `text-embeddings-ada-002`로 생성한 벡터를 검색하면 일치하는 키워드를 찾는데 얽매이지 않고 문장의 유사도만을 고려해서 검색한다. 오타도 적절하게 무시하면서 검색하는 것을 확인할 수 있다. 원하는 답변은 불교 경전의 특징에 대한 것이다.

In [ ]:
query = "최무가 간행에 기여한 경전에는 어떤 특징이 있나요?"  

docs = vector_store.similarity_search(
    query=query,
    k=3,
    search_type="similarity",
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)

## 2.1.1. 다국어 처리
`text-embeddings-ada-002`의 다국어 처리 능력을 확인하기 위해 한국어 문서를 영어로 검색해보자.

In [ ]:
query = "What are the characteristics of the sutras that Ch'oe U contributed to publication?"  

docs = vector_store.similarity_search(
    query=query,
    k=3,
    search_type="similarity",
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)

# 2.2. 하이브리드 검색
하이브리드 검색은 키워드 검색과 벡터 검색 모두를 쿼리에 사용한다. 키워드 검색 스코어를 구할 떄는 Okapi BM25 알고리즘을 사용해서 스코어를 계산하고, 벡터 검색에는 코사인 유사도를 기준으로 스코어를 계산한다. 이 서로 다른 계산 결과를 융합하는 방법으로는 [Reciprocal Rank Fusion(RRF)](https://learn.microsoft.com/azure/search/hybrid-search-ranking)을 사용한다. RRF는 두 방식으로 계산한 문서 랭크의 역수의 합을 구해서 문서 순위를 기준으로 어느 쪽의 랭크라도 상위에 있는 쪽이 스코어가 높아지는 구조다.

In [ ]:
query = "최우는 무신이었음에도 A와 B에 능숙했다. A와 B는 무엇일까?"  
# Perform a hybrid search
docs = vector_store.similarity_search(
    query=query,
    k=5,
    search_type="hybrid",
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)


In [ ]:
query = "최우는 무신이었음에도 A와 B에 능숙했다. A와 B는 무엇일까?"  
# similarity_search에 search_type="hybrid"를 지정하는 것과 동일하다.
docs = vector_store.hybrid_search(
    query=query,
    k=5
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)

# 2.3 시맨틱 하이브리드 검색
시맨틱 하이브리드 검색(하이브리드 검색 + 시맨틱 순위 책정)은 Azure AI Search에서만 지원하는 검색 기능으로, 하이브리드 검색과 검색 결과를 정확도가 높은 순서로 정렬하는 순위 재책정 기능(시맨틱 순위 책정)을 조합한 고도화된 검색 방법이다. 순위 재책정에는 마이크로소프트가 만든 언어 모델인 [Turing](https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/introducing-multilingual-support-for-semantic-search-on-azure/ba-p/2385110) 모델을 사용한다.

<div class="alert alert-block alert-success">
<b>Note:</b> LangChain의 Issues <a href="https://github.com/langchain-ai/langchain/issues/15355">#15355</a>가 해결될 때까지는 인덱스에 `metadata` 필드를 포함해야 한다.</div>

In [ ]:
query = "무신정변을 일으켜 무신정권을 수립한 무신 3명은?"  

docs = vector_store.semantic_hybrid_search(
    query=query,
    search_type="semantic_hybrid",
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)